#### Определения

In [ ]:
!pip install gensim
!pip install compress-fasttext

  Preparing metadata (setup.py) ... done
  Created wheel for compress-fasttext: filename=compress_fasttext-0.1.3-py3-none-any.whl size=14583 sha256=04cbc4dea0a926fb954d9155c32073c2cfb5b3e7ac7e5a974b6abefce065523a
  Stored in directory: /home/m_apishev/.cache/pip/wheels/c7/63/9f/39db0410175167cee5eeae4fde2405d957cd05c1d8811a51cf
Successfully built compress-fasttext


In [ ]:
from gensim import models

In [ ]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(x, y):
    return dot(x, y) / (norm(x) * norm(y))

print(cos_sim([1, 0, 0], (1, 0, 0)))
print(cos_sim([1, 0, 0], (0, 1, 0)))
print(cos_sim([1, 0, 0], (-1, 0, 0)))

1.0
0.0
-1.0


#### Обучим модель word2vec

In [ ]:
sentences = []
with open('rus-ru_web-public_2019_100K-sentences.txt') as fin:
    for line in fin:
        sentences.append(line.split(' '))

In [ ]:
model = models.Word2Vec()
model.build_vocab(sentences)
model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

(4079335, 6874035)

In [ ]:
try:
    print(cos_sim(model.wv['сеть'], model.wv['интернет']))
    print(cos_sim(model.wv['человек'], model.wv['интернет']))
    print(cos_sim(model.wv['интернет'], model.wv['интенет']))
except Exception as e:
    print(e)

0.9710866
0.5534083
"Key 'интенет' not present"


#### Обучим модель FastText

In [ ]:
model = models.FastText(sentences, sg=1, vector_size=50)

In [ ]:
try:
    print(cos_sim(model.wv['сеть'], model.wv['интернет']))
    print(cos_sim(model.wv['человек'], model.wv['интернет']))
    print(cos_sim(model.wv['интернет'], model.wv['интенет']))
except Exception as e:
    print(e)

0.60630214
0.3068741
0.87997293


#### Загрузим оптимизированную модель FastText

In [ ]:
import compress_fasttext
model = compress_fasttext.models.CompressedFastTextKeyedVectors.load(
    'https://github.com/avidale/compress-fasttext/releases/download/gensim-4-draft/geowac_tokens_sg_300_5_2020-100K-20K-100.bin'
)

In [ ]:
try:
    print(cos_sim(model['сеть'], model['интернет']))
    print(cos_sim(model['человек'], model['интернет']))
    print(cos_sim(model['интернет'], model['интенет']))
except Exception as e:
    print(e)

0.5231468445248225
0.27971834039337506
0.6044784950143409


## Домашнее задание
1. Скачать датасет с фразами симпосонов - https://github.com/sujanjoejacob/Text-mining-with-Simpsons-Data
2. Обучить FastText/Word2vec на фразах персонажей
3. Визуализировать embeddings по самым частотным словам (top 1000)
4. Найти самые близкие слова для:
  homer - marge + bart
  bart - lisa + school
  marge - homer + home
5. Попробовать построить классификатор bart/lisa с использованием этих эмбеддингов